In [37]:
import my_utils as mu
import pandas as pd
import json
from tqdm import tqdm

SELECT *
FROM (
    SELECT GAME_ID
    FROM RAWDATA
    WHERE PARTICIPANT_PUUID = (
        SELECT SUMMONER_PUUID
        FROM SUMMONER
        WHERE SUMMONER_NAME = '작은팽도리')
    ORDER BY GAME_ID DESC)
WHERE ROWNUM <= 20;

SELECT *
FROM RAWDATA
WHERE GAME_ID IN('KR_6488217131', 'KR_6488164039', 'KR_6488122352')
ORDER BY GAME_ID DESC;

요약: 게임 종료 시간, 승리 여부, 게임 총 시간, 사용 챔피언, 챔피언 레벨, 사용 스펠, 사용 룬 타입, 킬, 데스, 어시스트, kda, 아이템, 최다 연속킬, 킬 관여율, 제어 와드 설치 수, 총 cs, 분당 cs, 참여 소환사 이름 1~10번
(랭크 티어는 랭크 정보가 없어서 제거)

상세: 사용 챔피언, 챔피언 레벨, 사용 스펠, 사용 룬 타입, 소환사 이름, 킬, 데스, 어시스트, kda, 가한 피해량, 받은 피해량, 제어 와드 설치 수, 와드 설치 수, 와드 제거 수, 총 cs, 분당 cs, 아이템

In [58]:
mu.oracle_totalExecute('SELECT COUNT(*) FROM SUMMONER')

oracle open!
oracle close!


,COUNT(*)
0,183828


In [38]:
def rawdataLoader(datas: str):
    return json.loads(datas.replace("'", '"').replace('True', '1').replace('False', '0'))


def parseRawdata(rawdata: pd.DataFrame, parse_to: int, use_tqdm: bool=False) -> pd.DataFrame:
    """
    `parse_to`: 파싱 대상 컬럼의 시작 인덱스\n
    `use_tqdm`: tqdm 출력의 사용 여부
    """
    matches = {a:[] for a in rawdata.columns[parse_to:]}

    if use_tqdm: parse_data = tqdm(rawdata.iloc)
    else: parse_data = rawdata.iloc

    for rec in parse_data:
        for col in [a for a in rawdata.columns][parse_to:]:
            matches[col].append(rawdataLoader(rec[col]))
    for col in [a for a in rawdata.columns][parse_to:]:
        rawdata[col] = matches[col]
    return rawdata

In [51]:
sql = f"""
    SELECT *
    FROM (
        SELECT GAME_ID
        FROM RAWDATA
        WHERE PARTICIPANT_PUUID = (
            SELECT SUMMONER_PUUID
            FROM SUMMONER
            WHERE SUMMONER_NAME = '기본적으로못함')
        ORDER BY GAME_ID DESC)
    WHERE ROWNUM <= 20
"""
game_ids = mu.oracle_totalExecute(sql)

oracle open!
oracle close!


In [52]:
game_ids

,GAME_ID
0,KR_6484431751
1,KR_6484396115


In [53]:
game_ids['GAME_ID'].tolist().__str__()[1:-1]

"'KR_6484431751', 'KR_6484396115'"

In [56]:
sql = f"""
    SELECT ROWNUM NUM, r.*
    FROM RAWDATA r
    WHERE GAME_ID IN({game_ids['GAME_ID'].tolist().__str__()[1:-1]})
    ORDER BY GAME_ID DESC, PARTICIPANT_NUMBER
"""
game_datas = parseRawdata(mu.oracle_totalExecute(sql), 10)

oracle open!
oracle close!


In [ ]:
game_datas